In [2]:
import pandas as pd
import streamlit as st

trade = pd.read_csv('trade.csv')
countries = pd.read_csv('countries.csv')
commodities = pd.read_csv('commodities.csv')

In [3]:
trade.head()

,period,reporterCode,flowCode,partnerCode,cmdCode,primaryValue
0,2000,156,M,4,74,133.0
1,2000,156,M,4,71,10743.0
2,2000,156,M,8,60,1022.0
3,2000,156,M,12,27,23386009.0
4,2000,156,M,12,84,2.0


In [4]:
commodities.head()

,id,text,sector
0,1,Живые животные,Сельское хозяйство и пищевая промышленность
1,2,Мясо и пищевые мясные субпродукты,Сельское хозяйство и пищевая промышленность
2,3,"Рыба и ракообразные, моллюски и прочие водные ...",Сельское хозяйство и пищевая промышленность
3,4,Молочные продукты; яйца птиц; натуральный мёд;...,Сельское хозяйство и пищевая промышленность
4,5,"Продукты животного происхождения, не поименова...",Сельское хозяйство и пищевая промышленность


In [5]:
countries.head()

,id,text,reporterCodeIsoAlpha3,world_part
0,4,Афганистан,AFG,Азия
1,8,Албания,ALB,Европа
2,12,Алжир,DZA,Африка
3,20,Андорра,AND,Европа
4,24,Ангола,AGO,Африка


In [6]:
trade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548318 entries, 0 to 548317
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   period        548318 non-null  int64  
 1   reporterCode  548318 non-null  int64  
 2   flowCode      548318 non-null  object 
 3   partnerCode   548318 non-null  int64  
 4   cmdCode       548318 non-null  int64  
 5   primaryValue  548318 non-null  float64
dtypes: float64(1), int64(4), object(1)
memory usage: 25.1+ MB


In [7]:
commodities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      99 non-null     int64 
 1   text    99 non-null     object
 2   sector  99 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.4+ KB


In [8]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     256 non-null    object
 1   text                   256 non-null    object
 2   reporterCodeIsoAlpha3  256 non-null    object
 3   world_part             256 non-null    object
dtypes: object(4)
memory usage: 8.1+ KB


In [9]:
def camelcase_to_snakecase(name):
    import re
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()

trade.columns = [camelcase_to_snakecase(col) for col in trade.columns]
trade['partner_code'] = trade['partner_code'].astype(str)

print(trade.columns)

Index(['period', 'reporter_code', 'flow_code', 'partner_code', 'cmd_code',
       'primary_value'],
      dtype='object')


In [10]:
countries.columns = [camelcase_to_snakecase(col) for col in countries.columns]
countries = countries.rename(columns = {'text':'country_name'})
countries = countries.rename(columns = {'id':'country_id'})

commodities = commodities.rename(columns = {'text':'commodity_name'})
commodities = commodities.rename(columns = {'id':'commodity_id'})

print(countries.columns)
print(commodities.columns)

Index(['country_id', 'country_name', 'reporter_code_iso_alpha3', 'world_part'], dtype='object')
Index(['commodity_id', 'commodity_name', 'sector'], dtype='object')


In [11]:
countries[countries['country_id'] == '156']

,country_id,country_name,reporter_code_iso_alpha3,world_part
44,156,Китай,CHN,Азия


In [12]:
trade['reporter_code'] = 'Китай'

trade.head()

,period,reporter_code,flow_code,partner_code,cmd_code,primary_value
0,2000,Китай,M,4,74,133.0
1,2000,Китай,M,4,71,10743.0
2,2000,Китай,M,8,60,1022.0
3,2000,Китай,M,12,27,23386009.0
4,2000,Китай,M,12,84,2.0


In [13]:
trade = trade.merge(
    commodities,
    left_on='cmd_code',
    right_on='commodity_id',
    how='left'
)

trade = trade.drop(columns=['commodity_id'])

trade.head()

,period,reporter_code,flow_code,partner_code,cmd_code,primary_value,commodity_name,sector
0,2000,Китай,M,4,74,133.0,Медь и изделия из неё,Металлургия и металлообработка
1,2000,Китай,M,4,71,10743.0,"Драгоценные и полудрагоценные камни, драгоценн...",Минеральные материалы и изделия из них
2,2000,Китай,M,8,60,1022.0,Трикотажные и вязаные ткани,Текстильная и швейная промышленность
3,2000,Китай,M,12,27,23386009.0,"Минеральные топлива, масла и продукты их перег...",Топливно-энергетический комплекс
4,2000,Китай,M,12,84,2.0,"Ядерные реакторы, котлы, оборудование и механи...",Машиностроение и электроника


In [14]:
trade = trade.merge(
    countries[['country_id', 'country_name', 'world_part']],
    left_on='partner_code',
    right_on='country_id',
    how='left'
)

trade = trade.drop(columns=['country_id'])

trade.head()

,period,reporter_code,flow_code,partner_code,cmd_code,primary_value,commodity_name,sector,country_name,world_part
0,2000,Китай,M,4,74,133.0,Медь и изделия из неё,Металлургия и металлообработка,Афганистан,Азия
1,2000,Китай,M,4,71,10743.0,"Драгоценные и полудрагоценные камни, драгоценн...",Минеральные материалы и изделия из них,Афганистан,Азия
2,2000,Китай,M,8,60,1022.0,Трикотажные и вязаные ткани,Текстильная и швейная промышленность,Албания,Европа
3,2000,Китай,M,12,27,23386009.0,"Минеральные топлива, масла и продукты их перег...",Топливно-энергетический комплекс,Алжир,Африка
4,2000,Китай,M,12,84,2.0,"Ядерные реакторы, котлы, оборудование и механи...",Машиностроение и электроника,Алжир,Африка


In [15]:
trade = trade.rename(columns = {'country_name': 'partner_name', 'reporter_code': 'reporter_name'})

trade.head()

,period,reporter_name,flow_code,partner_code,cmd_code,primary_value,commodity_name,sector,partner_name,world_part
0,2000,Китай,M,4,74,133.0,Медь и изделия из неё,Металлургия и металлообработка,Афганистан,Азия
1,2000,Китай,M,4,71,10743.0,"Драгоценные и полудрагоценные камни, драгоценн...",Минеральные материалы и изделия из них,Афганистан,Азия
2,2000,Китай,M,8,60,1022.0,Трикотажные и вязаные ткани,Текстильная и швейная промышленность,Албания,Европа
3,2000,Китай,M,12,27,23386009.0,"Минеральные топлива, масла и продукты их перег...",Топливно-энергетический комплекс,Алжир,Африка
4,2000,Китай,M,12,84,2.0,"Ядерные реакторы, котлы, оборудование и механи...",Машиностроение и электроника,Алжир,Африка


In [16]:
trade = trade[['period', 'reporter_name', 'flow_code', 'partner_code', 'partner_name', 'world_part', 'cmd_code', 'commodity_name', 'sector', 'primary_value']]

trade.head()

,period,reporter_name,flow_code,partner_code,partner_name,world_part,cmd_code,commodity_name,sector,primary_value
0,2000,Китай,M,4,Афганистан,Азия,74,Медь и изделия из неё,Металлургия и металлообработка,133.0
1,2000,Китай,M,4,Афганистан,Азия,71,"Драгоценные и полудрагоценные камни, драгоценн...",Минеральные материалы и изделия из них,10743.0
2,2000,Китай,M,8,Албания,Европа,60,Трикотажные и вязаные ткани,Текстильная и швейная промышленность,1022.0
3,2000,Китай,M,12,Алжир,Африка,27,"Минеральные топлива, масла и продукты их перег...",Топливно-энергетический комплекс,23386009.0
4,2000,Китай,M,12,Алжир,Африка,84,"Ядерные реакторы, котлы, оборудование и механи...",Машиностроение и электроника,2.0


In [17]:
trade.to_csv('trade_merged.csv', index=False)